In [1]:
from sail.data import DataFrameGroup
from sail.algo import Fdlr
from sail.core import spawnvms
import optuna

In [2]:
# show two data servers.
vms = spawnvms(3)
vmagg = vms[-1]
vms = vms[:-1]

In [3]:
workplace = "/home/jjj/playground/tmp"

In [4]:
#DataFrameGroup is the collection of all dataset distributed among the VMs.
dfg = DataFrameGroup(vms, workplace)
dfg.import_data([0,0])

In [5]:
#dfg has a public data field shape, which depicts the shape of the datasets in rows and cols
dfg.shape

[(3, 6), (2, 6)]

In [6]:
# the col_label field has the fetures names of each col.
dfg.col_label

[Index(['Region', 'Temp(F)', 'Rainfall(mm)', 'Humidity(%)', 'Apples(ton)',
        'Oranges(ton)'],
       dtype='object'),
 Index(['Region', 'Temp(F)', 'Rainfall(mm)', 'Humidity(%)', 'Apples(ton)',
        'Oranges(ton)'],
       dtype='object')]

In [7]:
#make this part actual synthetic data
dfg.sample(vms[0], dfg.df[0])

,Region,Temp(F),Rainfall(mm),Humidity(%),Apples(ton),Oranges(ton)
0,Kanto,73,67,43,56,70
1,Johto,91,88,64,81,101
2,Hoenn,87,124,58,119,133


In [8]:
#feature engineering

In [9]:
#mu = dfg.mean('price')

In [10]:
#mu

In [11]:
#sigma = dfg.standard_deviation('price', mu)

In [12]:
#sigma

In [13]:
#sk = dfg.skew('price', mu, sigma)

In [14]:
#sk

In [15]:
#dfg.log_transform(['price'], ['log_price'])

In [16]:
#Defining the features and targets cols
#The get_col/to_numpy are done remotely
data = {}
X = dfg.get_col([['Temp(F)', 'Rainfall(mm)', 'Humidity(%)']]*len(vms), dfg.df)
y = dfg.get_col([['Apples(ton)', 'Oranges(ton)']]*len(vms), dfg.df)
data['X_train'] = dfg.to_numpy(X)
data['y_train'] = dfg.to_numpy(y)
#data['X_train'], data['X_test'], data['y_train'], data['y_test'] = dfg.train_test_split(X, y, 0.2, 42)

In [17]:
#construct the model
model = Fdlr(vms, vmagg, data, workplace)

In [18]:
def objective(trail, model):
    n_lr = trail.suggest_float('n_lr', 1e-5, 1e-4, step = 3e-4)
    model.initmodel(3,2,n_lr)
    model.fit(100)
    return model.score()

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, model), n_trials=3)

[I 2021-04-14 22:35:31,570] A new study created in memory with name: no-name-1d59695a-06fc-4066-8005-c09e37350614
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:547: UserWarning: The distribution is specified by [1e-05, 0.0001] and q=0.0003, but the range is not divisible by `q`. It will be replaced by [1e-05, 1e-05].
  warnings.warn(


processing round: 1
processing round: 10
processing round: 19
processing round: 28
processing round: 37
processing round: 46
processing round: 55
processing round: 64
processing round: 73
processing round: 82
processing round: 91
processing round: 100


[I 2021-04-14 22:39:27,319] Trial 0 finished with value: 12.906564394632975 and parameters: {'n_lr': 1e-05}. Best is trial 0 with value: 12.906564394632975.


processing round: 1
processing round: 10
processing round: 19
processing round: 28


In [16]:
#sigle round training
model.initmodel(3,2,n_lr)
#explain what fit is doing
model.fit(100)

In [ ]:
#Show prediction?

In [18]:
model.mae()

[1379.724577293396, 1373.2931090164184]